# Google Drive 활용
***

코드를 실행해보기 위해선 다음의 파이썬 라이브러리들이 설치되어있어야 한다.  
- google-api-python-client
- oauth2client

Anaconda Prompt에서 다음과 같은 명령어로 설치한다.

```sh
pip install google-api-python-client
pip install oauth2client
```

## 1. Google Drive API 사용설정
***


### 1.1. 프로젝트 선택 또는 새 프로젝트 생성
***

[Google API Console](https://console.developers.google.com/)에서 Google Drive API를 사용할 프로젝트를 선택한다.  
프로젝트가 없다면 다음과 같이 새 프로젝트를 만든다.

![새 프로젝트 생성](https://raw.githubusercontent.com/3x3x3/MediaFiles/master/google_drive/mk_project.gif)


### 1.2. Google Drive API 사용설정
***

![API 사용설정](https://raw.githubusercontent.com/3x3x3/MediaFiles/master/google_drive/enable_gd_api.gif)


### 1.3. 사용자 인증 정보 생성
***

서버에서 사용할 것이기 때문에 서비스 계정 키를 만든다.  
다운로드되는 서비스 계정 키는 다시 받을 수 없기 때문에 잘 보관하자.

![서비스 계정 키 생성](https://raw.githubusercontent.com/3x3x3/MediaFiles/master/google_drive/mk_service_key.gif)


### 1.4. Google Drive의 폴더에 권한설정
***

서비스 계정의 이메일로 사용할 폴더에 권한을 준다.

![폴더에 권한](https://raw.githubusercontent.com/3x3x3/MediaFiles/master/google_drive/set_auth_to_folder.gif)

### 1.5. 공통적으로 사용하는 코드
***

In [ ]:
# -*- coding: utf-8 -*-
import io
import pprint
from googleapiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials
from httplib2 import Http

# 다운 받은 서비스 계정 키의 파일명을 service_acc_key.json로 변경하거나 아래 코드를 변경
SERVICE_ACC_KEY_FILE_NAME = "service_acc_key.json"

# SCOPES에 들어가는 값들은 다음 URL을 참조: https://developers.google.com/drive/api/v3/about-auth
SCOPES = [
    "https://www.googleapis.com/auth/drive"
]

credentials = ServiceAccountCredentials.from_json_keyfile_name(SERVICE_ACC_KEY_FILE_NAME, SCOPES)
http_auth = credentials.authorize(Http())
drive = build("drive", "v3", http=http_auth)

## 2. 읽기
***


### 2.1. 특정 파일/폴더의 ID를 알아내기
***

![파일/폴더의 ID 알아내기](https://raw.githubusercontent.com/3x3x3/MediaFiles/master/google_drive/get_file_id.gif)



### 2.2. 특정 폴더 내 모든 폴더 및 파일들의 MetaData를 출력
***

In [ ]:
FOLDER_ID = "1kBvzgj9MIuDLNqN0cVZhR-wDDfvbKXH4"

response = drive.files().list(
    q="'{0}' in parents".format(FOLDER_ID),
    spaces="drive",
    fields="files(id, name, mimeType, parents)"
).execute()

files = response.get("files", [])

print("\n##### List of Metadata #####\n")
pprint.pprint(files)

### 2.3. Paging
***
폴더내의 목록을 불러올 때 기본적으로 100개까지 불러오게 되며, pageSize에 값을 줘서 한 번에 조회되는 개수를 조절할 수 있다. (1 ~ 1000개)  
조회하려는 목록이 한 번에 조회되는 목록보다 많을 때는 Paging 처리를 해야 한다.

In [ ]:
FOLDER_ID = "17lrAKEA73zf_EAuimoMmyebVAg-8p2GZ"
page_token = None

while True:
    print("\n##### List of Metadata #####\n")

    response = drive.files().list(
        q="'{0}' in parents".format(FOLDER_ID),
        spaces="drive",
        fields="nextPageToken, files(id, name, mimeType, parents)",
        pageSize=3,  # 한 번에 조회할 개수
        pageToken=page_token,
        orderBy="name",
    ).execute()

    files = response.get("files", [])
    pprint.pprint(files)

    print("\n##### nextPageToken #####\n")

    page_token = response.get('nextPageToken', None)
    print(page_token)

    if page_token is None:
        break;

### 2.4. 폴더/파일의 ID를 이용하여 Metadata를 출력
***

In [ ]:
FILE_ID = "1EFzqg-bfKXOjHP3q7NNUM6QYuEln8sR5"  # 샘플로 읽어볼 파일의 ID
FOLDER_ID = "1kBvzgj9MIuDLNqN0cVZhR-wDDfvbKXH4"  # 샘플로 읽어볼 폴더의 ID

# fields에 들어가는 인자는 다음 URL을 참조: https://developers.google.com/drive/api/v3/reference/files
FIELDS = "kind, id, name, mimeType, size"

print("\n##### File's MetaData #####\n")
response = drive.files().get(fileId=FILE_ID, fields=FIELDS).execute()
pprint.pprint(response)

print("\n\n##### Folder's MetaData #####\n")
response = drive.files().get(fileId=FOLDER_ID, fields=FIELDS).execute()
pprint.pprint(response)

### 2.5. 파일 다운로드
***

In [ ]:
from googleapiclient.http import MediaIoBaseDownload

FILE_ID = "1EFzqg-bfKXOjHP3q7NNUM6QYuEln8sR5"  # 다운로드 받을 파일의 ID

print("\n##### Download Start #####\n")
request = drive.files().get_media(fileId=FILE_ID)
buffer = io.BytesIO()
downloader = MediaIoBaseDownload(buffer, request)
done = False

while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))

print("\n\n##### Download Result #####\n")
print(buffer.getvalue())

## 3. 쓰기
***

### 3.1. 파일 업로드
***

In [ ]:
from googleapiclient.http import MediaFileUpload

FOLDER_ID = "1nWQXXoOGoOIsEJk5GLvUmVqWACWy53Ho"  # 여기에 업로드

meta_data = {
    "name": "file_up_test.txt",
    "parents": [FOLDER_ID],
}

uploader = MediaFileUpload("upload_test.txt", mimetype="text/plain")
response = drive.files().create(body=meta_data, media_body=uploader).execute()

print("\n\n##### Upload Result #####\n")
print(response)

### 3.2. 메모리 업로드
***

In [ ]:
from googleapiclient.http import MediaIoBaseUpload

FOLDER_ID = "1nWQXXoOGoOIsEJk5GLvUmVqWACWy53Ho"  # 여기에 업로드

meta_data = {
    "name": "mem_up_test.txt",
    "parents": [FOLDER_ID],
}

buffer = io.BytesIO()
buffer.write(b"hello world!")

uploader = MediaIoBaseUpload(fd=buffer, mimetype="text/plain")
response = drive.files().create(
    body=meta_data, 
    media_body=uploader
).execute()

print("\n\n##### Upload Result #####\n")
print(response)

## 4. 수정
***

In [ ]:
FILE_ID = response["id"]  # 위에서 메모리 업로드로 Create된 파일의 ID

meta_data = {
    "name": "mem_up_test2.txt",
}

buffer = io.BytesIO()
buffer.write(b"hi")

uploader = MediaIoBaseUpload(fd=buffer, mimetype="text/plain")
response = drive.files().update(
    fileId=FILE_ID,
    body=meta_data, 
    media_body=uploader
).execute()

print("\n##### Upload Result #####\n")
print(response)

## 5. 삭제
***

In [ ]:
response = drive.files().delete(fileId=FILE_ID).execute()

print("\n##### Delete Result #####\n")

if 0 == len(response):
    print("delete complete")
else
    print("failed to delete")